# Recurrent Neural Network Character-Level Language Model

This is the "working notebook", with code to load and train the model, as well as run unit tests.

In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz

# Your code
import rnnlm; reload(rnnlm)
import rnnlm_test; reload(rnnlm_test)

import json
import pandas as pd
from glob import glob
import csv

In [2]:
#tf.reset_default_graph()

#wordid_ph = tf.placeholder(tf.int32, shape=[None, None])
#embedding_matrix = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]])
#x = tf.nn.embedding_lookup(embedding_matrix, wordid_ph)

#print('wordid placeholder shape:', wordid_ph.shape)
#print('x shape:', x.shape)

#sess = tf.Session()
## Two sentences, each with four words.
#wordids = [[1, 2, 1, 2], [0, 0, 0, 0]]
#x_val = sess.run(x, feed_dict={wordid_ph: wordids})
#print('Embeddings shape:', x_val.shape)  # 2 sentences, 4 words, 
#print('Embeddings value:\n', x_val)

wordid placeholder shape: (?, ?)
x shape: (?, ?, 3)
Embeddings shape: (2, 4, 3)
Embeddings value:
 [[[2 2 2]
  [3 3 3]
  [2 2 2]
  [3 3 3]]

 [[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]]


### Tensorboard

Load implementation, construct graph, and write a logdir for TensorBoard.
```
cd assignment/a4
tensorboard --logdir /tmp/w266/a4_graph --port 6006
```
http://localhost:6006/

In [2]:
reload(rnnlm)

TF_GRAPHDIR = "/tmp/artificial_hotel_reviews/a4_graph"

# Clear old log directory.
shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

lm = rnnlm.RNNLM(V=10000, H=200, num_layers=2)
lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildSamplerGraph()

summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [4]:
#with lm.graph.as_default():
    #initializer = tf.global_variables_initializer()

#with tf.Session(graph=lm.graph) as session:
    #session.run(initializer)
    #variables_names = [v.name for v in tf.trainable_variables()]
    #values = session.run(variables_names)
    #for k, v in zip(variables_names, values):
        #print("Variable: ", k)
        #print("Shape: ", v.shape)
        #print(v)

Variable:  W_in:0
Shape:  (10000, 200)
[[-0.78748155 -0.05848813 -0.64172769 ...,  0.84141541 -0.94063973
   0.6318059 ]
 [ 0.73271012  0.9191277   0.16166019 ..., -0.77998519  0.349545
  -0.45933056]
 [ 0.56432152 -0.03329587  0.20995927 ..., -0.3828721   0.12824035
  -0.13679957]
 ..., 
 [-0.12901855 -0.29448342 -0.08786106 ..., -0.73254704 -0.37692833
  -0.20111871]
 [-0.62848496 -0.50970197  0.9452734  ...,  0.99917698 -0.81992722
  -0.37649083]
 [-0.09145284  0.00680947  0.67164588 ..., -0.92668557  0.46153927
   0.74277234]]
Variable:  rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0
Shape:  (400, 800)
[[ 0.02885439 -0.00585808 -0.04069098 ...,  0.00699186 -0.05404399
   0.02202466]
 [ 0.06509807 -0.02223223  0.02077565 ..., -0.00037639  0.04783005
  -0.00804034]
 [ 0.02742642  0.05705827 -0.03898749 ...,  0.00316557  0.06507974
  -0.01836298]
 ..., 
 [-0.00754741 -0.04928695  0.06990113 ..., -0.06414328 -0.06926721
  -0.05963309]
 [-0.06505661 -0.0187963   0.05553678 ..., -0.0

**Unit Test**

In [2]:
reload(rnnlm); reload(rnnlm_test)
utils.run_tests(rnnlm_test, ["TestRNNLMCore", "TestRNNLMTrain", "TestRNNLMSampler"])

test_shapes_embed (rnnlm_test.TestRNNLMCore) ... ok
test_shapes_output (rnnlm_test.TestRNNLMCore) ... ok
test_shapes_recurrent (rnnlm_test.TestRNNLMCore) ... ok
test_shapes_train (rnnlm_test.TestRNNLMTrain) ... 

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



ok
test_shapes_sample (rnnlm_test.TestRNNLMSampler) ... ok

----------------------------------------------------------------------
Ran 5 tests in 3.315s

OK


## Train RNNLM

Data loader functions in **`utils.py`**.

`utils.rnnlm_batch_generator` returns an iterator that yields minibatches in the correct format. Batches will be of size `[batch_size, max_time]`, and consecutive batches will line up along rows so that the final state $h^{\text{final}}$ of one batch can be used as the initial state $h^{\text{init}}$ for the next.

Toy corpus example:

In [6]:
toy_corpus = "<s> Mary had a little lamb . <s> The lamb was white as snow . <s>"
toy_corpus = np.array(toy_corpus.split())

html = "<h3>Input words w:</h3>"
html += "<table><tr><th>Batch 0</th><th>Batch 1</th></tr><tr>"
bi = utils.rnnlm_batch_generator(toy_corpus, batch_size=2, max_time=4)
for i, (w,y) in enumerate(bi):
    cols = ["w_%d" % d for d in range(w.shape[1])]
    html += "<td>{:s}</td>".format(utils.render_matrix(w, cols=cols, dtype=object))
html += "</tr></table>"
display(HTML(html))

html = "<h3>Target words y:</h3>"
html += "<table><tr><th>Batch 0</th><th>Batch 1</th></tr><tr>"
bi = utils.rnnlm_batch_generator(toy_corpus, batch_size=2, max_time=4)
for i, (w,y) in enumerate(bi):
    cols = ["y_%d" % d for d in range(y.shape[1])]
    html += "<td>{:s}</td>".format(utils.render_matrix(y, cols=cols, dtype=object))
html += "</tr></table>"
display(HTML(html))

### run_epoch()

`train=True` flag enables train mode. `train=False` runs `score_dataset()`.

In [2]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=None):
    assert(learning_rate is not None)
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation

    for i, (w, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})

        #### YOUR CODE HERE ####
        feed_dict = {lm.input_w_:w,
                     lm.target_y_:y,
                     lm.learning_rate_: learning_rate,
                     lm.use_dropout_: use_dropout,
                     lm.initial_h_:h}
        cost, h, _ = session.run([loss, lm.final_h_, train_op],feed_dict=feed_dict)

        #### END(YOUR CODE) ####
        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print("[batch {:d}]: seen {:d} words at {:.1f} wps, loss = {:.3f}".format(
                i, total_words, avg_wps, avg_cost))
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [3]:
def score_dataset(lm, session, ids, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils.rnnlm_batch_generator(ids, batch_size=100, max_time=100)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=0.0, train=False, 
                     verbose=False, tick_s=3600)
    print("{:s}: avg. loss: {:.03f}  (perplexity: {:.02f})".format(name, cost, np.exp(cost)))
    return cost

**Training Unit Test**

In [5]:
reload(rnnlm); reload(rnnlm_test)
th = rnnlm_test.RunEpochTester("test_toy_model")
th.setUp(); th.injectCode(run_epoch, score_dataset)
unittest.TextTestRunner(verbosity=2).run(th)

test_toy_model (rnnlm_test.RunEpochTester) ... 

[batch 0]: seen 50 words at 43.1 wps, loss = 2.028
[batch 98]: seen 4950 words at 2284.2 wps, loss = 1.114
[batch 210]: seen 10550 words at 3329.2 wps, loss = 0.959
[batch 346]: seen 17350 words at 4155.3 wps, loss = 1.024
[batch 481]: seen 24100 words at 4656.4 wps, loss = 0.983
[batch 615]: seen 30800 words at 4987.1 wps, loss = 0.943
[batch 751]: seen 37600 words at 5237.5 wps, loss = 0.954
[batch 887]: seen 44400 words at 5425.6 wps, loss = 0.964
[batch 1024]: seen 51250 words at 5579.2 wps, loss = 0.954
[batch 1160]: seen 58050 words at 5697.4 wps, loss = 0.943
[batch 1296]: seen 64850 words at 5794.1 wps, loss = 0.930
[batch 1431]: seen 71600 words at 5872.2 wps, loss = 0.921
[batch 1567]: seen 78400 words at 5940.1 wps, loss = 0.917
[batch 1702]: seen 85150 words at 5995.7 wps, loss = 0.905
[batch 1839]: seen 92000 words at 6050.8 wps, loss = 0.909
[batch 1976]: seen 98850 words at 6098.0 wps, loss = 0.914
[batch 2112]: seen 105650 words at 6138.3 wps, loss = 0.906
[batch 2245]:

ok

----------------------------------------------------------------------
Ran 1 test in 33.626s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### Training

At the end of training, `tf.train.Saver` saves a copy of the model to `/tmp/w266/artificial_hotel_reviews/rnnlm_trained`. To load this from disk, see **part (d)** for an example of how this is done.

#### Tuning Hyperparameters

In [10]:
## Load the dataset
#V = 10000
#vocab, train_ids, test_ids = utils.load_corpus("brown", split=0.8, V=V, shuffle=42)

[nltk_data] Downloading package brown to /home/kalvin_kao/nltk_data...
[nltk_data]   Package brown is already up-to-date!
Vocabulary: 10,000 types
Loaded 57,340 sentences (1.16119e+06 tokens)
Training set: 45,872 sentences (924,077 tokens)
Test set: 11,468 sentences (237,115 tokens)


In [4]:
review_path = '/home/kalvin_kao/yelp_challenge_dataset/review.csv'
business_path = '/home/kalvin_kao/yelp_challenge_dataset/business.csv'

In [5]:
review_df = pd.read_csv(review_path)

In [6]:
business_df = pd.read_csv(business_path)

In [7]:
five_star_review_df = review_df[review_df['stars']==5]
five_star_review_series = five_star_review_df['text']

In [8]:
#build a list of list of characters from the 5-star reviews
def preprocess_review_series(review_series):
    review_list = []
    for new_review in five_star_review_series:
        clipped_review = new_review[2:-1]
        char_list = list(clipped_review.lower())
        semifinal_review = []
        last_char = ''
        for ascii_char in char_list:
            if ascii_char == '\\' or last_char == '\\':
                pass
            else:
                semifinal_review.append(ascii_char)
            last_char = ascii_char
        final_review = ['<SOR>'] + semifinal_review + ['<EOR>']
        #print(final_review)
        review_list.append(final_review)
    return review_list

In [9]:
#preprocessed reviews
#review_list = preprocess_review_series(five_star_review_series)
review_list = preprocess_review_series(five_star_review_series)

In [16]:
#combined_review_list = [item for sublist in review_list for item in sublist]
training_review_list = [item for sublist in review_list[:20000] for item in sublist]
print(len(training_review_list))
test_review_list = [item for sublist in review_list[20000:21000] for item in sublist]
#unique_characters = list(set(combined_review_list))
unique_characters = list(set(training_review_list + test_review_list))
#len(unique_characters)

10406707


In [17]:
#vocabulary
char_dict = {w:i for i, w in enumerate(unique_characters)}
ids_to_words = {v: k for k, v in char_dict.items()}
#print(char_dict)
#print(ids_to_words)

In [18]:
print(len(char_dict.keys()))
print(len(unique_characters))

67
67


In [19]:
#convert to flat (1D) np.array(int) of ids
#add memory to VM and remove 1000 slice
#combined_review_ids = [char_dict.get(token) for token in combined_review_list[:1000]]
training_review_ids = [char_dict.get(token) for token in training_review_list]
test_review_ids = [char_dict.get(token) for token in test_review_list]

In [20]:
train_ids = np.array(training_review_ids)
test_ids = np.array(test_review_ids)

In [21]:
# Training parameters
## add parameter sets for each attack/defense configuration
#max_time = 25
max_time = 100
#batch_size = 100
batch_size = 256
#learning_rate = 0.01
learning_rate = 0.002
#num_epochs = 10
num_epochs = 1

# Model parameters
#model_params = dict(V=vocab.size, 
                    #H=200, 
                    #softmax_ns=200,
                    #num_layers=2)
model_params = dict(V=len(unique_characters), 
                    H=1024, 
                    softmax_ns=len(unique_characters),
                    num_layers=2)

TF_SAVEDIR = "/tmp/artificial_hotel_reviews/a4_model"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")

In [22]:
# Will print status every this many seconds
print_interval = 5

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)
    
    #check trainable variables
    #variables_names = [v.name for v in tf.trainable_variables()]
    #values = session.run(variables_names)
    #for k, v in zip(variables_names, values):
        #print("Variable: ", k)
        #print("Shape: ", v.shape)
        #print(v)

    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        bi = utils.rnnlm_batch_generator(train_ids, batch_size, max_time)
        print("[epoch {:d}] Starting epoch {:d}".format(epoch, epoch))
        #### YOUR CODE HERE ####
        # Run a training epoch.
        run_epoch(lm, session, batch_iterator=bi, train=True, verbose=True, tick_s=10, learning_rate=learning_rate)

        
        #### END(YOUR CODE) ####
        print("[epoch {:d}] Completed in {:s}".format(epoch, utils.pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print("[epoch {:d}]".format(epoch), end=" ")
        score_dataset(lm, session, train_ids, name="Train set")
        print("[epoch {:d}]".format(epoch), end=" ")
        score_dataset(lm, session, test_ids, name="Test set")
        print("")
    
    # Save final model
    saver.save(session, trained_filename)
    
    #variables_names = [v.name for v in tf.trainable_variables()]
    #values = session.run(variables_names)
    #for k, v in zip(variables_names, values):
        #print("Variable: ", k)
        #print("Shape: ", v.shape)
        #print(v)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

[epoch 1] Starting epoch 1
[batch 0]: seen 25600 words at 1064.6 wps, loss = 4.762
[batch 1]: seen 51200 words at 1117.5 wps, loss = 10.085
[batch 2]: seen 76800 words at 1137.5 wps, loss = 11.354
[batch 3]: seen 102400 words at 1146.2 wps, loss = 11.476
[batch 4]: seen 128000 words at 1149.0 wps, loss = 10.534
[batch 5]: seen 153600 words at 1153.5 wps, loss = 10.008
[batch 6]: seen 179200 words at 1156.0 wps, loss = 9.317
[batch 7]: seen 204800 words at 1158.1 wps, loss = 8.666
[batch 8]: seen 230400 words at 1159.9 wps, loss = 8.107
[batch 9]: seen 256000 words at 1161.1 wps, loss = 7.629
[batch 10]: seen 281600 words at 1162.9 wps, loss = 7.217
[batch 11]: seen 307200 words at 1163.7 wps, loss = 6.858
[batch 12]: seen 332800 words at 1164.4 wps, loss = 6.545
[batch 13]: seen 358400 words at 11

[batch 132]: seen 3404800 words at 1168.3 wps, loss = 2.519
[batch 133]: seen 3430400 words at 1168.3 wps, loss = 2.513
[batch 134]: seen 3456000 words at 1168.4 wps, loss = 2.508
[batch 135]: seen 3481600 words at 1168.3 wps, loss = 2.503
[batch 136]: seen 3507200 words at 1168.3 wps, loss = 2.498
[batch 137]: seen 3532800 words at 1168.4 wps, loss = 2.493
[batch 138]: seen 3558400 words at 1168.4 wps, loss = 2.488
[batch 139]: seen 3584000 words at 1168.5 wps, loss = 2.483
[batch 140]: seen 3609600 words at 1168.5 wps, loss = 2.478
[batch 141]: seen 3635200 words at 1168.5 wps, loss = 2.474
[batch 142]: seen 3660800 words at 1168.5 wps, loss = 2.469
[batch 143]: seen 3686400 words at 1168.5 wps, loss = 2.464
[batch 144]: seen 3712000 words at 1168.5 wps, loss = 2.459
[batch 145]: seen 3737600 words at 1168.5 wps, loss = 2.455
[batch 146]: seen 3763200 words at 1168.5 wps, loss = 2.450
[batch 147]: seen 3788800 words at 1168.6 wps, loss = 2.446
[batch 148]: seen 3814400 words at 1168.

[batch 269]: seen 6912000 words at 1170.5 wps, loss = 2.098
[batch 270]: seen 6937600 words at 1170.6 wps, loss = 2.096
[batch 271]: seen 6963200 words at 1170.6 wps, loss = 2.094
[batch 272]: seen 6988800 words at 1170.6 wps, loss = 2.093
[batch 273]: seen 7014400 words at 1170.6 wps, loss = 2.091
[batch 274]: seen 7040000 words at 1170.7 wps, loss = 2.089
[batch 275]: seen 7065600 words at 1170.7 wps, loss = 2.087
[batch 276]: seen 7091200 words at 1170.7 wps, loss = 2.085
[batch 277]: seen 7116800 words at 1170.7 wps, loss = 2.084
[batch 278]: seen 7142400 words at 1170.7 wps, loss = 2.082
[batch 279]: seen 7168000 words at 1170.7 wps, loss = 2.080
[batch 280]: seen 7193600 words at 1170.7 wps, loss = 2.078
[batch 281]: seen 7219200 words at 1170.8 wps, loss = 2.077
[batch 282]: seen 7244800 words at 1170.8 wps, loss = 2.075
[batch 283]: seen 7270400 words at 1170.8 wps, loss = 2.073
[batch 284]: seen 7296000 words at 1170.8 wps, loss = 2.072
[batch 285]: seen 7321600 words at 1170.

[batch 406]: seen 10406656 words at 1171.7 wps, loss = 1.914
[epoch 1] Completed in 2:28:01
[epoch 1] Train set: avg. loss: 1.373  (perplexity: 3.95)
[epoch 1] Test set: avg. loss: 1.378  (perplexity: 3.97)



## Sampling Sentences

In [23]:
def sample_step(lm, session, input_w, initial_h):
    """Run a single RNN step and return sampled predictions.
  
    Args:
      lm : rnnlm.RNNLM
      session: tf.Session
      input_w : [batch_size] vector of indices
      initial_h : [batch_size, hidden_dims] initial state
    
    Returns:
      final_h : final hidden state, compatible with initial_h
      samples : [batch_size, 1] vector of indices
    """
    # Reshape input to column vector
    input_w = np.array(input_w, dtype=np.int32).reshape([-1,1])
  
    #### YOUR CODE HERE ####
    # Run sample ops
    #lm.BuildSamplerGraph()
    feed_dict = {lm.input_w_:input_w, lm.initial_h_:initial_h}
    final_h, samples = session.run([lm.final_h_, lm.pred_samples_], feed_dict=feed_dict)


    #### END(YOUR CODE) ####
    # Note indexing here: 
    #   [batch_size, max_time, 1] -> [batch_size, 1]
    return final_h, samples[:,-1,:]

In [24]:
# Same as above, but as a batch
#max_steps = 20
max_steps = 50
num_samples = 10
random_seed = 42

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildSamplerGraph()

with lm.graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(random_seed)
    
    # Load the trained model
    saver.restore(session, trained_filename)

    # Make initial state for a batch with batch_size = num_samples
    #w = np.repeat([[vocab.START_ID]], num_samples, axis=0)
    [char_dict.get(token) for token in test_review_list]
    w = np.repeat([[char_dict.get('<SOR>')]], num_samples, axis=0)
    h = session.run(lm.initial_h_, {lm.input_w_: w})
    # We'll take one step for each sequence on each iteration 
    for i in range(max_steps):
        h, y = sample_step(lm, session, w[:,-1:], h)
        w = np.hstack((w,y))

    # Print generated sentences
    for row in w:
        for i, word_id in enumerate(row):
            #print(vocab.id_to_word[word_id], end=" ")
            print(ids_to_words[word_id], end="")
            #if (i != 0) and (word_id == vocab.START_ID):
            if (i != 0) and (word_id == char_dict.get("<EOR>")):
                break
        print("")

INFO:tensorflow:Restoring parameters from /tmp/artificial_hotel_reviews/a4_model/rnnlm_trained
<SOR>the trom and i pick in lass-hot. the shows when ho
<SOR>wings for my busy wrop feeling will back of the br
<SOR>strone (maymoriandure dethy the gengroulder. close
<SOR>here after the feels on eating attentive place and
<SOR>is always upment you parked but that a pretty but 
<SOR>special" it-was customer places...oh everyone that
<SOR>highly recommend the down had. i ordered this only
<SOR>is always clust on you can taste chicken! definite
<SOR>cut - more meal favorite complaints, she serve sho
<SOR>if you must car) charg for sandwich this mented us


## Linguistic Properties

Given two or more test sentences, the model should score the more plausible (or more correct) sentence with a higher log-probability.

In [15]:
def score_seq(lm, session, seq, vocab):
    """Score a sequence of words. Returns total log-probability."""
    padded_ids = vocab.words_to_ids(utils.canonicalize_words(["<s>"] + seq + ["</s>"], 
                                                             wordset=vocab.word_to_id))
    w = np.reshape(padded_ids[:-1], [1,-1])
    y = np.reshape(padded_ids[1:],  [1,-1])
    h = session.run(lm.initial_h_, {lm.input_w_: w})
    feed_dict = {lm.input_w_:w,
                 lm.target_y_:y,
                 lm.initial_h_:h,
                 lm.dropout_keep_prob_: 1.0}
    # Return log(P(seq)) = -1*loss
    return -1*session.run(lm.loss_, feed_dict)

def load_and_score(inputs, sort=False):
    """Load the trained model and score the given words."""
    lm = rnnlm.RNNLM(**model_params)
    lm.BuildCoreGraph()
    
    with lm.graph.as_default():
        saver = tf.train.Saver()

    with tf.Session(graph=lm.graph) as session:  
        # Load the trained model
        saver.restore(session, trained_filename)

        if isinstance(inputs[0], str) or isinstance(inputs[0], bytes):
            inputs = [inputs]

        # Actually run scoring
        results = []
        for words in inputs:
            score = score_seq(lm, session, words, vocab)
            results.append((score, words))

        # Sort if requested
        if sort: results = sorted(results, reverse=True)

        # Print results
        for score, words in results:
            print("\"{:s}\" : {:.02f}".format(" ".join(words), score))

**Example Test**

In [16]:
#sents = ["once upon a time",
         #"the quick brown fox jumps over the lazy dog"]
#load_and_score([s.split() for s in sents])

INFO:tensorflow:Restoring parameters from /tmp/artificial_hotel_reviews/a4_model/rnnlm_trained
"once upon a time" : -8.74
"the quick brown fox jumps over the lazy dog" : -7.83


In [17]:
#sents = ["is sentence nonsense this", "i drive a car"]
#load_and_score([s.split() for s in sents])

In [18]:
#sents = ["the boy and the girl are", "the boy and the girl is"]
#load_and_score([s.split() for s in sents])

INFO:tensorflow:Restoring parameters from /tmp/artificial_hotel_reviews/a4_model/rnnlm_trained
"the boy and the girl are" : -6.03
"the boy and the girl is" : -5.90


In [19]:
#sents = ["peanuts are my favorite kind of nut",
         #"peanuts are my favorite kind of vegetable"]
#load_and_score([s.split() for s in sents])

#sents = ["when I'm hungry I really prefer to eat",
         #"when I'm hungry I really prefer to drink"]
#load_and_score([s.split() for s in sents])

INFO:tensorflow:Restoring parameters from /tmp/artificial_hotel_reviews/a4_model/rnnlm_trained
"peanuts are my favorite kind of nut" : -7.46
"peanuts are my favorite kind of vegetable" : -7.29
INFO:tensorflow:Restoring parameters from /tmp/artificial_hotel_reviews/a4_model/rnnlm_trained
"when I'm hungry I really prefer to eat" : -7.49
"when I'm hungry I really prefer to drink" : -7.52


In [20]:
#prefix = "I have lots of".split()
#noun = "toys"
#adjectives = ["square", "green", "plastic"]
#inputs = []
#for adjs in itertools.permutations(adjectives):
    #words = prefix + list(adjs) + [noun]
    #inputs.append(words)
    
#load_and_score(inputs, sort=True)

INFO:tensorflow:Restoring parameters from /tmp/artificial_hotel_reviews/a4_model/rnnlm_trained
"I have lots of plastic green square toys" : -8.51
"I have lots of green square plastic toys" : -8.56
"I have lots of green plastic square toys" : -8.57
"I have lots of plastic square green toys" : -8.64
"I have lots of square green plastic toys" : -8.69
"I have lots of square plastic green toys" : -8.71
